In [1]:
from pycbc.catalog import Merger
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.frame import read_frame
from pycbc.filter import highpass, resample_to_delta_t
from pycbc.inference import models, sampler
from pycbc.inference.sampler import emcee_pt
from pycbc.distributions import Uniform, JointDistribution, SinAngle, UniformRadius, External, Gaussian
from IPython.display import Image
import matplotlib.pyplot as plt
import copy
import pylab
import numpy as np
import bilby

mname = "GW170814"
m = Merger(mname)

In [ ]:
data_config = """
[data]
instruments = H1 L1 V1
trigger-time = {event_tc}
analysis-start-time = -8
analysis-end-time = 2
psd-estimation = median-mean
psd-start-time = -256
psd-end-time = 256
psd-segment-length = 8
psd-segment-stride = 4
psd-inverse-length = 8
strain-high-pass = 15
pad-data = 8
sample-rate = 2048
frame-files = H1:data/{merger}_H1_4KHZ.gwf L1:data/{merger}_L1_4KHZ.gwf V1:data/{merger}_V1_4KHZ.gwf
channel-name = H1:GWOSC-4KHZ_R1_STRAIN L1:GWOSC-4KHZ_R1_STRAIN V1:GWOSC-4KHZ_R1_STRAIN
""".format(event_tc=m.time, merger=mname)

prior_config = """
[model]
name = gaussian_noise
low-frequency-cutoff = 20.0

[variable_params]
delta_tc =
mass1 =
mass2 =
distance =
coa_phase =
inclination =
polarization =
ra =
dec =

[static_params]
approximant = IMRPhenomPv2
f_lower = 20
f_ref = 20
trigger_time = ${data|trigger-time}

[prior-delta_tc]
name = uniform
min-delta_tc = -0.1
max-delta_tc = 0.1

[waveform_transforms-tc]
name = custom
inputs = delta_tc
tc = ${data|trigger-time} + delta_tc

[prior-mass1]
name = uniform
min-mass1 = 20.
max-mass1 = 35.

[prior-mass2]
name = uniform
min-mass2 = 20.
max-mass2 = 35.

[prior-distance]
name = uniform_radius
min-distance = 10
max-distance = 1000

[prior-coa_phase]
name = uniform_angle

[prior-inclination]
name = sin_angle

[prior-ra+dec]
name = uniform_sky

[prior-polarization]
name = uniform_angle
"""

sampler_config = """
[sampler]
name = emcee_pt
nwalkers = 200
ntemps = 20
effective-nsamples = 1000
checkpoint-interval = 1000
max-samples-per-chain = 1000

[sampler-burn_in]
burn-in-test = nacl & max_posterior

[sampling_params]
mass1, mass2 : mchirp, q

[sampling_transforms-mchirp+q]
name = mass1_mass2_to_mchirp_q
"""

!echo '{data_config}' > pycbc_conf/data.ini
!echo '{prior_config}' > pycbc_conf/prior.ini
!echo '{sampler_config}' > pycbc_conf/sampler.ini

In [ ]:
!pycbc_inference --verbose --config-files pycbc_conf/data.ini pycbc_conf/prior.ini pycbc_conf/sampler.ini --output-file pycbc_conf/inference.hdf --nprocesses 14 --force   

In [ ]:
!pycbc_inference_plot_posterior --verbose --input-file pycbc_conf/inference.hdf --output-file pycbc_conf/post_corner.png \
    --plot-density --plot-contours --plot-marginal --parameters ra dec mass1 mass2 distance

In [ ]:
Image('pycbc_conf/post_corner.png')

In [2]:
ifos = ['H1', 'V1', 'L1']

data_filenames = {}
psds = {}
data = {}

for ifo in ifos:
    print("Processing {} data".format(ifo))
    
    fname = 'data/{}_{}_4KHZ.gwf'.format(mname, ifo)
    data_filenames[ifo] = fname

    ts = read_frame(fname, "{}:GWOSC-4KHZ_R1_STRAIN".format(ifo),
                    start_time=int(m.time - 260),
                    end_time=int(m.time + 40))
    ts = highpass(ts, 15.0)                     # Remove low frequency content
    ts = resample_to_delta_t(ts, 1.0/2048)      # Resample data to 2048 Hz
    ts = ts.time_slice(m.time-112, m.time + 16) # Limit to times around the signal
    data[ifo] = ts.to_frequencyseries()         # Convert to a frequency series by taking the data's FFT

    # Estimate the power spectral density of the data
    psd = interpolate(ts.psd(4), ts.delta_f)
    psd = inverse_spectrum_truncation(psd, int(4 * psd.sample_rate), 
                                      trunc_method='hann',
                                      low_frequency_cutoff=20.0)
    psds[ifo] = psd

Processing H1 data
Processing V1 data
Processing L1 data


In [3]:
static = {'mass1':30.6,
          'mass2':25.2,
          'f_lower':25,
          'approximant':"TaylorF2",
          'polarization':0,
         }

variable = ('distance',
            'inclination',
            'ra',
            'dec',
            'tc')

inclination_prior = SinAngle(inclination=None)
distance_prior = UniformRadius(distance=(100, 1000))
tc_prior = Uniform(tc=(m.time-0.1, m.time+0.1))

hp = bilby.gw.prior.HealPixMapPriorDist('data/GW170814_skymap.fits.gz')
hp_ra_prior = bilby.gw.prior.HealPixPrior(hp, 'ra')
hp_dec_prior = bilby.gw.prior.HealPixPrior(hp, 'dec')

def invcdf_ra(pr=hp_ra_prior, **kwds):
    updated = {}
    print(hp_ra_prior.rescale([0.4, .3, .2]))
    updated['ra'] = pr.rescale(kwds['ra'])
    return updated

def logpdf_ra(pr=hp_ra_prior, **kwargs):
    return pr.ln_prob(kwargs['ra'])

def invcdf_dec(pr=hp_dec_prior, **kwds):
    updated = {}
    updated['dec'] = pr.rescale(kwds['dec'])
    return updated

def logpdf_dec(pr=hp_dec_prior, **kwargs):
    return pr.ln_prob(kwargs['dec'])

ra_prior = External(['ra'], logpdf_ra, cdfinv=invcdf_ra)
dec_prior = External(['dec'], logpdf_dec, cdfinv=invcdf_dec)

prior = JointDistribution(variable, inclination_prior, distance_prior, ra_prior, dec_prior, tc_prior)

In [4]:
gauss_prior = Gaussian(noise=None, noise_mean=0, noise_var=1)
bilby_gauss = bilby.core.prior.Gaussian(0, 1, 'noise')

def invcdf_g(pr=bilby_gauss, **kwds):
    updated = {}
    for param in kwds:
        updated[param] = pr.rescale(kwds[param])
    return updated

def logpdf_g(pr=bilby_gauss, **kwds):
    updated = {}
    for param in kwds:
        updated[param] = pr.ln_prob(kwds[param])
    return updated

bg_prior = External(['noise'], logpdf_g, cdfinv=invcdf_g)

In [6]:
val = 0.4

x = gauss_prior.cdfinv(noise=val)
y = bg_prior.cdfinv(noise=val)
print(x, y)

val = [0.1, 0.2, 0.3]

x = gauss_prior.logpdf(noise=val)
print(x)
y = bg_prior.logpdf(noise=val)
print(y)

{'noise': -0.2533471031357998} {'noise': -0.2533471031357998}


TypeError: '<=' not supported between instances of 'ClosedBound' and 'list'

In [ ]:
model = models.SingleTemplate(variable, copy.deepcopy(data),
                              low_frequency_cutoff={'H1':25, 'L1':25, 'V1':25},
                              psds = psds,
                              static_params = static,
                              prior = prior,
                              sample_rate = 8192,
                              )

smpl = sampler.emcee_pt.EmceePTSampler(model, 3, 200, nprocesses=12)
_ = smpl.set_p0() # If we don't set p0, it will use the models prior to draw initial points!emcee_pt.

In [ ]:
smpl.run_mcmc(200)

In [ ]:
lik = smpl.model_stats['loglikelihood']
s = smpl.samples

# Note how we have to access the arrays differently that before since there is an additional dimension. 
# The zeroth element of that dimension represents the 'coldest' and is the one we want for our results.
# The other temperatures represent a modified form of the likelihood that allows walkers to traverse
# the space more freely.
pylab.scatter(s['distance'][0,:,-1],
              s['inclination'][0,:,-1],
              c=lik[0,:,-1])
pylab.xlabel('Distance (Mpc)')
pylab.ylabel('Inclination (Radians)')

c = pylab.colorbar()
c.set_label('Loglikelihood')
pylab.show()